In [ ]:
!arch -x86_64 pip3 install scikit-learn
# Install SciPy
!arch -x86_64 pip3 install datetime
!arch -x86_64 pip3 install scipy
# Install MySQL connector for Python
!arch -x86_64 pip3 install mysql-connector-python
!arch -x86_64 pip3 install pymc
!arch -x86_64 pip3 install pandas

In [ ]:
!arch -x86_64 pip3 install scikit-learn
# Install SciPy
!arch -x86_64 pip3 install datetime
!arch -x86_64 pip3 install scipy
# Install MySQL connector for Python
!arch -x86_64 pip3 install mysql-connector-python
!arch -x86_64 pip3 install pymc
!arch -x86_64 pip3 install pandas

from sklearn import linear_model
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix
import mysql.connector
from datetime import datetime
import pandas as pd

df = pd.read_csv("/Users/timotaij/Downloads/HW2NBADATA/HW2/Name_and_IDs.csv")
player_dict = df.set_index('PERSON_ID')['DISPLAY_LAST_COMMA_FIRST'].to_dict()


def calculate_decay_weights(dates, decay_rate=0.005):
    # Assume dates are already datetime.date objects
    most_recent_date = max(dates)
    days_ago = [(most_recent_date - date).days for date in dates]
    decay_weights = np.exp(-decay_rate * np.array(days_ago))
    return decay_weights

def run_ridge_model(X, y, sample_weights):
    clf = linear_model.RidgeCV(alphas=[1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 3750, 4000,4250,4500,4750,5000], cv=6)
    clf.fit(X, y, sample_weight=sample_weights)
    print('ALPHA:', clf.alpha_)
    return clf.coef_

def replace_player_ids_with_names():
    db = mysql.connector.connect(host="*******", user="****", passwd="*******", db="********")
    cur = db.cursor()
    cur.execute("SELECT home_poss, pts, a1, a2, a3, a4, a5, h1, h2, h3, h4, h5, date FROM matchups WHERE season BETWEEN 2018 AND 2024")
    data = cur.fetchall()
    db.close()

    # Replace player IDs with names directly after fetching the data
    processed_data = []
    for item in data:
        new_item = list(item)
        for i in range(2, 12):  # Replace player IDs in a1 to h5 columns
            player_id = new_item[i]
            player_name = player_dict.get(int(player_id), "UnknownPlayer")
            new_item[i] = player_name
        processed_data.append(tuple(new_item))

    return processed_data

def main():
    processed_data = replace_player_ids_with_names()



    all_players, dates = {}, []
    for item in processed_data:
        for i in range(2, 12):
            all_players[item[i]] = 1
        dates.append(item[12])

    # Calculate decay weights based on dates
    decay_weights = calculate_decay_weights(dates)

    player_to_col = {}
    col_to_player = {}
    for p in all_players:
        for side in ['off', 'def']:
            p_side = str(p) + '_' + side
            if p_side not in player_to_col:
                number = len(player_to_col)
                player_to_col[p_side] = number
                col_to_player[number] = p_side
   
    X = lil_matrix((len(processed_data), len(col_to_player)))
    y = np.zeros(len(processed_data))
    for i, item in enumerate(processed_data):
        home_poss = item[0]
        pts = item[1]

        home_list, away_list = [], []
        for j in range(2, 7):
            away_list.append(item[j])
        for j in range(7, 12):
            home_list.append(item[j])
        
        off_list, def_list = (home_list, away_list) if home_poss else (away_list, home_list)
        for p in off_list:
            X[i, player_to_col[str(p) + '_off']] = 1
        for p in def_list:
            X[i, player_to_col[str(p) + '_def']] = 1
        
        y[i] = pts

    y_av = np.average(y)
    print("Hello")
    beta_ridge = run_ridge_model(X.tocsr(), y - y_av, decay_weights)
    beta_ridge = beta_ridge * 100
    print("Hello2")
    # Lists to hold DataFrame columns
    players = []
    offense = []
    defense = []

# Extract player names and corresponding scores in pairs
    for idx in range(14, len(col_to_player), 2):  # Starting index 14 here as your example
        off_player = col_to_player[idx]
        def_player = col_to_player[idx + 1]
    
        player_name = off_player[:-4]  # Remove '_off' to get the base player name
        players.append(player_name)
    
        offense_score = beta_ridge[idx]
        defense_score = beta_ridge[idx + 1]
    
        offense.append(offense_score)
        defense.append(defense_score)

# Creating the DataFrame
    df = pd.DataFrame({
        "Player": players,
        "Offense": offense,
        "Defense": defense
    })
    df['Ovr'] = df['Offense'] - df['Defense']

# Sort DataFrame by 'Ovr' in descending order
    df_sorted = df.sort_values(by='Defense', ascending=True)
    pd.set_option('display.max_rows', None)

# Display or return the sorted DataFrame
    print(df_sorted)


    

if __name__ == '__main__':
    main()

